# See input.sx for a sample SPHInX input that I would like to achieve

In [1]:
import numpy as np
from pyiron import Project
from pyiron.sphinx.structure import read_atoms

In [115]:
pr = Project("sphinx_input")
test = pr.create_job(pr.job_type.Sphinx, "test")
test.structure = read_atoms("structure.sx")

2020-04-07 15:49:02,792 - pyiron_log - WARNING - Job aborted - please remove it and run again! test


# Currently:
What I have below doesn't work. It ultimately writes the _guess_str dictionary literally to guess.sx.
Normally I just overwrite the file, e.g. `with open("guess.sx", "w")`... immediately
after I submit the job, which cannot be the best way.

In [116]:
test._guess_str = {
    "waves": {
        "lcao": {
            "maxSteps": 1,
            "rhoMixing": 0.
        },
        "pawBasis": None
    },
    "rho": {
        "atomicOrbitals": None,
        "charged": {
            "charge": 2,
            "z": 20
        }
    }
}

#test.run()

# It would be great to arbitrarily add/overwrite parameters with dictionaries:

I think if the primary groups that should be tunable in SPHInX (basis, PAWHamiltonian, and initialGuess) were initialized as full dictionaries with their respective defaults, this would be a straightforward way to customize them later, as Sam suggested.

And if they are meant to be externally modifiable, they should probably not carry an underscore prefix.

In [1]:
import os
from pyiron import Project

In [2]:
os.chdir("/Users/ashton/pyiron/projects")
pr = Project("sphinx_test")
job = pr.create_job(pr.job_type.Sphinx, "test")
job.structure = pr.create_structure("Fe", "bcc", 2.78)

In [3]:
job.input.guess["rho"]["charged"] = {"charge": 2, "z": 20}

job.input.guess.rho.charged.charge = 3
job.input.hamilton.set_flag("dipoleCorrection")
job.input.hamilton.set("nExcessElectrons", -2)

job.input.hamilton.nExcessElectrons = -4
job.write_input()
print(job.input.basis)

/Users/ashton/pyiron/projects/sphinx_test/test_hdf5/test
{'eCut': 'EnCut', 'kPoint': {'coords': 'KpointCoords', 'weight': 1, 'relative': True}, 'folding': 'KpointFolding', 'saveMemory': True}


/Users/ashton/Software/sphinx_input/pyiron/sphinx/base.py:368: UserWarning: executable version could not be identified
  warnings.warn("executable version could not be identified")


In [8]:
!cat /Users/ashton/pyiron/projects/sphinx_test/test_hdf5/test/PAWhamiltonian.sx

nEmptyStates = 5;
ekt = Sigma;
xc = Xcorr;
dipoleCorrection;
nExcessElectrons = -4;


In [6]:
!cat /Users/ashton/pyiron/projects/sphinx_test/test_hdf5/test/userparameters.sx

EnCut=340;
KpointCoords=[0.5, 0.5, 0.5];
KpointFolding=[4, 4, 4];
EmptyStates=auto;
Sigma=0.2;
Xcorr=PBE;
VaspPot=True;
Estep=400;
Ediff=0.0001;
WriteWaves=True;
KJxc=False;
SaveMemory=True;
CoarseRun=False;
rhoMixing=1.0;
spinMixing=1.0;
CheckOverlap=True;
THREADS=1;
